In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone.utils.eval.detection import evaluate_detections, DetectionResults
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from numpyencoder import NumpyEncoder
import json

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import time
from codecarbon import track_emissions



# List available zoo datasets
print(foz.list_zoo_datasets())

#
# Load the COCO-2017 validation split into a FiftyOne dataset
#
# This will download the dataset from the web, if necessary
#
dataset = foz.load_zoo_dataset("coco-2017", split="validation")
#test17 = foz.load_zoo_dataset("coco-2017", split="test")

# Give the dataset a new name, and make it persistent so that you can
# work with it in future sessions
dataset.name = "coco-2017-validatio"
dataset.persistent = True
#test17.name = "coco-2017-test-example"
#test17.persistent = True
# Visualize the in the App
#session = fo.launch_app(dataset)

['activitynet-100', 'activitynet-200', 'bdd100k', 'caltech101', 'caltech256', 'cifar10', 'cifar100', 'cityscapes', 'coco-2014', 'coco-2017', 'fashion-mnist', 'fiw', 'hmdb51', 'imagenet-2012', 'imagenet-sample', 'kinetics-400', 'kinetics-600', 'kinetics-700', 'kinetics-700-2020', 'kitti', 'kitti-multiview', 'lfw', 'mnist', 'open-images-v6', 'open-images-v7', 'quickstart', 'quickstart-geo', 'quickstart-groups', 'quickstart-video', 'sama-coco', 'ucf101', 'voc-2007', 'voc-2012']
Found annotations at '/Users/kristian/fiftyone/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [9.8s elapsed, 0s remaining, 537.9 samples/s]       
Dataset 'coco-2017-validation' created


In [70]:
model_url = "https://www.kaggle.com/models/tensorflow/ssd-mobilenet-v2/frameworks/TensorFlow2/variations/ssd-mobilenet-v2/versions/1"
model = hub.load(model_url)

num_images_to_infer = 3000
sampled_dataset = dataset.take(num_images_to_infer)
coco_results = []

# Iterate through each sample in the dataset
for sample in sampled_dataset:
    coco_sample_results = []

    # Load the image
    image_path = sample.filepath
    image = tf.image.decode_image(tf.io.read_file(image_path), channels=3).numpy()
    image_height, image_width, _ = image.shape
    image_id = int(image_path[-16:-4])

    # Perform inference
    input_image = tf.convert_to_tensor(image)
    input_image = tf.expand_dims(input_image, axis=0)
    input_image = tf.cast(input_image, tf.uint8) 
    detections = model(input_image)

    # Extract relevant information from the detections
    boxes = detections["detection_boxes"][0].numpy()
    scores = detections["detection_scores"][0].numpy()
    labels = detections["detection_classes"][0].numpy().astype(int)

    # Append image id to results
    for box, score, label in zip(boxes, scores, labels):
        # Convert to COCO format
        coco_sample_results.append(
            {
                "image_id": image_id,
                "category_id": label,
                # Scale box to image size
                "bbox":  [
                    box[1] * image_width,
                    box[0] * image_height,
                    (box[3] - box[1]) * image_width,
                    (box[2] - box[0]) * image_height,
                ],
                "score": score,
            }
        )

    coco_results.extend(coco_sample_results)

In [71]:
# Save the results to disk
coco_results_path = "coco_results.json"
with open(coco_results_path, "w") as f:
    json.dump(coco_results, f, cls=NumpyEncoder)

# Load the COCO annotations for the validation split
gt_path = 'data/coco/annotations/instances_val2017.json'
coco_gt = COCO(gt_path)

# Load the COCO results
coco_dt = coco_gt.loadRes(coco_results_path)

# Evaluate the results
coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
#coco_eval.params.imgIds  = imgIds
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()


loading annotations into memory...
Done (t=0.85s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.23s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=11.23s).
Accumulating evaluation results...
DONE (t=2.64s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.216
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.257
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.131
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.201
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

In [37]:
session = fo.launch_app(sampled_dataset)